In [315]:
# Add module import path to system path.
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Hot-loading modules before executing code: allows us to write code while testing it in this notebook.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [316]:
import logging

# Logging strategy for this notebook.
root = logging.getLogger()
root.setLevel(logging.INFO)

if not root.hasHandlers():
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)

    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)

    root.addHandler(handler)

In [317]:
from parser import Parser

parser = Parser()

In [318]:
seasons, teams = parser.parse()

2022-03-13 11:47:48,344 - parser - INFO - Column names are TeamID, TeamName, FirstD1Season, LastD1Season
2022-03-13 11:47:48,346 - parser - INFO - Processed 372 teams.
2022-03-13 11:47:48,347 - parser - INFO - Column names are Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, NumOT
2022-03-13 11:47:48,866 - parser - INFO - Processed 174471 games.
2022-03-13 11:47:48,867 - parser - INFO - Column names are Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, NumOT
2022-03-13 11:47:48,873 - parser - INFO - Processed 2317 games.
2022-03-13 11:47:48,875 - parser - INFO - Column names are Season, Seed, TeamID
2022-03-13 11:47:48,879 - parser - INFO - Processed 2354 seeds.
2022-03-13 11:47:48,880 - parser - INFO - Column names are Season, DayZero, RegionW, RegionX, RegionY, RegionZ
2022-03-13 11:47:49,103 - parser - INFO - Processed 38 seasons.


In [319]:
train_seasons = []
for year in range(1985, 2016):
    train_seasons.append(seasons[year])

test_seasons = []
for year in range(2016, 2020):
    test_seasons.append(seasons[year])

In [320]:
from span import Span

train_span = Span(train_seasons)
test_span = Span(test_seasons)

In [321]:
span_predictions = test_span.predict()
scores = Span.score(span_predictions)

AttributeError: 'Tournament' object has no attribute 'tournament'

In [313]:
scores

{2016: 0.6931471805599453,
 2017: 0.6931471805599453,
 2018: 0.6931471805599453,
 2019: 0.6931471805599453,
 'Average': 0.6931471805599453}

In [314]:
def get_scores(max_iter):
    classifier = train_span.train(max_iter)
    classifiers = test_span.build_seasons_classifiers_map(classifier)
    span_predictions = test_span.predict(classifiers)
    scores = Span.score(span_predictions)
    return scores

get_scores(50000)

{2016: 0.5661946065837112,
 2017: 0.5363595561086407,
 2018: 0.6162056151202538,
 2019: 0.500544058493409,
 'Average': 0.5548259590765037}

In [238]:
import numpy as np
from matplotlib import pyplot as plt

d = {}
for x in range(1, 100000, 10000):
    d[x] = get_average_score(x)


lists = sorted(d.items()) # sorted by key, return a list of tuples

x, y = zip(*lists) # unpack a list of pairs into two tuples

plt.plot(x, y)
plt.show()

NameError: name 'get_average_score' is not defined

In [343]:
# Build UI that lets you pick the features to use. 

# Train n times and give average score + min/max score per year and total over the span of years used for validating.

{2016: 0.5841837309797322,
 2017: 0.5270035679014219,
 2018: 0.6269825819043252,
 2019: 0.517863598005087,
 -1: 0.5640083696976416}

In [289]:
[prediction.probability for prediction in span_predictions[2017]]

[0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156045891,
 0.5019441156

In [ ]:
# Teams with good defense: allow less points
# 3 point PCT
# Look at coaches